In [4]:
import spotipy
import config
from spotipy.oauth2 import SpotifyClientCredentials

**Authentication to use Spotify's API**

In [5]:
client_credentials_manager = SpotifyClientCredentials(client_id = config.client_ID, client_secret = config.client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

**Function to extract Song's URIs from Playlist's URL**


In [20]:
def extract_URI(playlist_URL):
    playlist_URI = playlist_URL.split("/")[-1].split("?")[0]
    track_URIs = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]
    return playlist_URI, track_URIs

**Extracting the URIs from Top 50 Global Songs**

In [21]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
playlist_URI, track_URIs = extract_URI(playlist_link)
#URIs

**Extract Track's Data from Playlist's URIs**

In [76]:
def extract_playlist_song_data(playlist_URL):
    playlist_URI, track_URIs = extract_URI(playlist_URL)
    
    playlist_data = {'Track Name' : [], 'Track URI' : [], 'Track Album' : [], 'Track Popularity' : [], 
                    'Artists Name' : [], 'Artists URI' : [], 'Artists Popularity' : [], 'Artists Genres' : []}
    
    playlist_tracks = sp.playlist_tracks(playlist_URI)
    for track in playlist_tracks["items"]:
        #Track info
        playlist_data['Track Name'].append(track["track"]["name"])
        playlist_data['Track URI'].append(track["track"]["uri"])
        playlist_data['Track Album'].append(track["track"]["album"]["name"])
        playlist_data['Track Popularity'].append(track["track"]["popularity"])
        
        #Artists Info
        artists = track["track"]["artists"]
        artists_uri = {}
        artists_name = []
        artists_popul = {}
        artists_genres = {}
        
        for artist in artists:
            artists_name.append(artist['name'][0])
            artists_uri.update({artist['name'] : artist['uri']})
            artist_info = sp.artist(artist['uri'])
            artists_popul.update({artist['name'] : artist_info["popularity"]})
            artists_genres.update({artist['name'] : artist_info["genres"]})
                        
        playlist_data['Artists Name'].append(artists_name)
        playlist_data['Artists URI'].append(artists_uri)
        playlist_data['Artists Popularity'].append(artists_popul)
        playlist_data['Artists Genres'].append(artists_genres)
    
    return playlist_data
        

**Visualization of collected Data as a DataFrame**

In [81]:
import pandas as pd

playlist_data = extract_playlist_song_data(playlist_link)
tracks_df = pd.DataFrame(playlist_data)

In [80]:
tracks_df.head()

,Track Name,Track URI,Track Album,Track Popularity,Artists Name,Artists URI,Artists Popularity,Artists Genres
0,un x100to,spotify:track:6pD0ufEQq0xdHSsRbg9LBK,un x100to,99,"[Grupo Frontera, Bad Bunny]",{'Grupo Frontera': 'spotify:artist:6XkjpgcEsYa...,"{'Grupo Frontera': 88, 'Bad Bunny': 97}","{'Grupo Frontera': ['musica chihuahuense', 'nu..."
1,Ella Baila Sola,spotify:track:3qQbCzHBycnDpGskqOWY0E,DESVELADO,87,"[Eslabon Armado, Peso Pluma]",{'Eslabon Armado': 'spotify:artist:0XeEobZplHx...,"{'Eslabon Armado': 86, 'Peso Pluma': 94}","{'Eslabon Armado': ['corrido', 'corridos tumba..."
2,La Bebe - Remix,spotify:track:2UW7JaomAMuX9pZrjVpHAU,La Bebe (Remix),99,"[Yng Lvcas, Peso Pluma]",{'Yng Lvcas': 'spotify:artist:1NNRWkhwmcXRimFY...,"{'Yng Lvcas': 81, 'Peso Pluma': 94}","{'Yng Lvcas': [], 'Peso Pluma': ['corrido', 'n..."
3,Cupid - Twin Ver.,spotify:track:7FbrGaHYVDmfr7KoLIZnQ7,The Beginning: Cupid,97,[FIFTY FIFTY],{'FIFTY FIFTY': 'spotify:artist:4GJ6xDCF5jaUqD...,{'FIFTY FIFTY': 81},{'FIFTY FIFTY': ['k-pop girl group']}
4,Flowers,spotify:track:4DHcnVTT87F0zZhRPYmZ3B,Endless Summer Vacation,90,[Miley Cyrus],{'Miley Cyrus': 'spotify:artist:5YGY8feqx7naU7...,{'Miley Cyrus': 89},{'Miley Cyrus': ['pop']}


**Extracting Metadata and Features from Track's URI**

In [116]:
def extract_track_features(track_name, track_URI):
    features = sp.audio_features(track_URI)
    return {track_name : features[0]}

def extract_playlist_features(playlist_URL):
    playlist_URI, tracks_URI = extract_URI(playlist_URL)
    
    playlist_features = {}
    playlist_tracks = sp.playlist_tracks(playlist_URI)["items"]
    playlist_names = [track["track"]["name"] for track in playlist_tracks]
    
    for name, track in zip(playlist_names, track_URIs):
        playlist_features.update(extract_track_features(name, track))
        
    return playlist_features

In [117]:
features_dict = extract_playlist_features(playlist_link)
features_df = pd.DataFrame(features_dict).T
features_df = features_df.reset_index().rename(columns={'index': 'Track Name'})

In [118]:
features_df.head()

,Track Name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,un x100to,0.569,0.724,6,-4.076,0,0.0474,0.228,0,0.27,0.562,83.118,audio_features,6pD0ufEQq0xdHSsRbg9LBK,spotify:track:6pD0ufEQq0xdHSsRbg9LBK,https://api.spotify.com/v1/tracks/6pD0ufEQq0xd...,https://api.spotify.com/v1/audio-analysis/6pD0...,194563,4
1,Ella Baila Sola,0.668,0.758,5,-5.176,0,0.0332,0.483,0.000019,0.0837,0.834,147.989,audio_features,3qQbCzHBycnDpGskqOWY0E,spotify:track:3qQbCzHBycnDpGskqOWY0E,https://api.spotify.com/v1/tracks/3qQbCzHBycnD...,https://api.spotify.com/v1/audio-analysis/3qQb...,165671,3
2,La Bebe - Remix,0.812,0.479,2,-5.678,0,0.333,0.213,0.000001,0.0756,0.559,169.922,audio_features,2UW7JaomAMuX9pZrjVpHAU,spotify:track:2UW7JaomAMuX9pZrjVpHAU,https://api.spotify.com/v1/tracks/2UW7JaomAMuX...,https://api.spotify.com/v1/audio-analysis/2UW7...,234353,4
3,Cupid - Twin Ver.,0.783,0.592,11,-8.332,0,0.0331,0.435,0.000004,0.347,0.726,120.018,audio_features,7FbrGaHYVDmfr7KoLIZnQ7,spotify:track:7FbrGaHYVDmfr7KoLIZnQ7,https://api.spotify.com/v1/tracks/7FbrGaHYVDmf...,https://api.spotify.com/v1/audio-analysis/7Fbr...,174253,4
4,Flowers,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,audio_features,4DHcnVTT87F0zZhRPYmZ3B,spotify:track:4DHcnVTT87F0zZhRPYmZ3B,https://api.spotify.com/v1/tracks/4DHcnVTT87F0...,https://api.spotify.com/v1/audio-analysis/4DHc...,200455,4
